In [1]:
from datetime import datetime, time, timedelta
import math
import statistics
import random
random.seed(datetime.now().timestamp())
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 1)
import matplotlib.pyplot as plt

In [2]:
def random_roll():
    last_time = datetime.now().timestamp()
    while True:
        time_now = datetime.now().timestamp()
        if last_time != time_now:
            random.seed(time_now)
            last_time = time_now
        else:
            random.seed(random.random())
        yield random.random()
rnd = random_roll()

class Model:
    def __init__(self, config):
        self.attributes = ["arrival_time","service_span","service_start","system_wait","queue_span","elimination_time","eliminated","queue_length"]
        self.config = config
        i_accumulator = 0
        for i in range(len(self.config.incoming_distr)):
            i_accumulator += self.config.incoming_distr[i][1]
            self.config.incoming_distr[i].append(i_accumulator)
        self.counter = 0
        self.store = []
        self.queue = []
        self.arrival_time = 0
    def random_gen(self):
        for i in range(len(self.config.incoming_distr)):
            if self.config.incoming_distr[i][2] >= next(rnd):
                self.arrival_time += self.config.incoming_distr[i][0][0] + next(rnd) * self.config.incoming_distr[i][0][1]
                break
        self.service_span = self.config.service_span_from + self.config.service_span_range * next(rnd)
        self.queue_span = self.config.queue_span_from + self.config.queue_span_range * next(rnd)
    def time_increment(self):
        self.counter += 1
        self.random_gen()
        service_start = self.arrival_time + self.queue_span
        system_wait_span = self.service_span + self.queue_span
        eliminated = ""
        eliminated_list = []
        for i in self.queue:
            if i[1] < self.arrival_time:
                eliminated_list.append(i)
                eliminated += f'{i[0]} '
            else:
                break
        for i in eliminated_list:
            self.queue.remove(i)
        elimination_time = self.arrival_time + system_wait_span
        self.queue.append([self.counter, elimination_time])
        return [
            self.arrival_time,
            self.service_span,
            service_start,
            system_wait_span,
            self.queue_span,
            elimination_time,
            eliminated,
            len(self.queue)-1]
    def model(self, timespan = 3650):
        for _ in range(timespan):
            self.store.append(self.time_increment())
        df = pd.DataFrame(self.store, columns=self.attributes)
        display(df)
        return df

class Object:
    def __init__(self, **attributes):
        self.__dict__.update(attributes)

In [3]:
config = Object(
    incoming_distr = [
        [[1,1], 0.05],
        [[2,10], 0.2],
        [[11,20], 0.2],
        [[21,30], 0.3],
        [[31,40], 0.1],
        [[40,60], 0.15]],
    queue_count = 1,
    queue_span_from = 1,
    queue_span_range = 10,
    service_span_from = 1,
    service_span_range = 8,
    start_hour = 8)

model = Model(config)
df = model.model(50)

,arrival_time,service_span,service_start,system_wait,queue_span,elimination_time,eliminated,queue_length
0,5.5,8.9,6.5,9.9,1.0,15.4,,0
1,36.4,7.6,45.8,17.0,9.4,53.4,1,0
2,37.7,8.6,45.8,16.8,8.2,54.5,,1
3,66.6,2.4,67.7,3.5,1.1,70.1,2 3,0
4,92.4,2.9,98.6,9.1,6.2,101.5,4,0
5,104.8,4.3,108.0,7.6,3.3,112.3,5,0
6,139.3,5.5,141.5,7.7,2.3,147.0,6,0
7,157.9,4.7,165.4,12.2,7.5,170.1,7,0
8,174.9,4.8,185.5,15.4,10.6,190.3,8,0
9,213.7,3.3,218.3,8.0,4.6,221.6,9,0


In [118]:
df["arrival_time"].diff()

0      NaN
1     47.5
2     19.2
3      8.5
4     37.1
5     92.7
6     30.1
7     31.1
8     32.8
9      1.6
10    11.7
11    15.8
12     1.1
13     3.1
14    28.4
15    67.4
16    50.8
17     9.8
18    33.8
19    27.5
20     6.6
21    47.3
22    23.5
23     6.6
24     4.1
25    22.3
26    22.4
27    14.3
28    15.8
29    42.0
30    26.0
31    25.0
32    28.1
33     3.5
34    32.8
35    15.4
36     1.5
37    47.8
38    30.4
39     4.0
40    31.8
41     7.2
42    43.3
43    47.8
44    14.1
45     8.6
46    49.9
47    22.6
48    49.7
49    11.5
Name: arrival_time, dtype: float64

In [4]:
def to_time(minutes, start=datetime(2024, 3, 30, 8)):
    timestamp_minutes = math.floor(minutes)
    timestamp_seconds = math.floor((minutes - timestamp_minutes) * 60)
    timestamp = start + timedelta(minutes=timestamp_minutes, seconds=timestamp_seconds)
    return timestamp

time_update = lambda x: to_time(x).strftime('%H:%M:%S')
df["arrival_time"] = df["arrival_time"].apply(time_update)
df["service_start"] = df["service_start"].apply(time_update)
df["elimination_time"] = df["elimination_time"].apply(time_update)

In [5]:
display(df)

,arrival_time,service_span,service_start,system_wait,queue_span,elimination_time,eliminated,queue_length
0,08:05:29,8.9,08:06:29,9.9,1.0,08:15:24,,0
1,08:36:25,7.6,08:45:49,17.0,9.4,08:53:26,1,0
2,08:37:40,8.6,08:45:50,16.8,8.2,08:54:28,,1
3,09:06:33,2.4,09:07:40,3.5,1.1,09:10:05,2 3,0
4,09:32:22,2.9,09:38:37,9.1,6.2,09:41:29,4,0
5,09:44:45,4.3,09:48:01,7.6,3.3,09:52:19,5,0
6,10:19:15,5.5,10:21:32,7.7,2.3,10:26:59,6,0
7,10:37:53,4.7,10:45:23,12.2,7.5,10:50:06,7,0
8,10:54:52,4.8,11:05:31,15.4,10.6,11:10:18,8,0
9,11:33:39,3.3,11:38:16,8.0,4.6,11:41:36,9,0


In [6]:
config = Object(
    count = 5,
    length = int(len(df)/5),
    total = [],
    mean = [],
    total_temp = [],
    mean_temp = [])
for i in range(config.count):
    pos = (i * config.length)
    data_slice = df.iloc[pos:(pos+config.length)]['system_wait']
    config.total_temp.extend([0]*config.length)
    config.total_temp[-1] = data_slice.sum()
    config.mean.append(data_slice.mean())
    config.mean_temp.extend([0]*config.length)
    config.mean_temp[-1] = data_slice.mean()
df['interval_total'] = config.total_temp
df['interval_mean'] = config.mean_temp
display(df)

,arrival_time,service_span,service_start,system_wait,queue_span,elimination_time,eliminated,queue_length,interval_total,interval_mean
0,08:05:29,8.9,08:06:29,9.9,1.0,08:15:24,,0,0.0,0.0
1,08:36:25,7.6,08:45:49,17.0,9.4,08:53:26,1,0,0.0,0.0
2,08:37:40,8.6,08:45:50,16.8,8.2,08:54:28,,1,0.0,0.0
3,09:06:33,2.4,09:07:40,3.5,1.1,09:10:05,2 3,0,0.0,0.0
4,09:32:22,2.9,09:38:37,9.1,6.2,09:41:29,4,0,0.0,0.0
5,09:44:45,4.3,09:48:01,7.6,3.3,09:52:19,5,0,0.0,0.0
6,10:19:15,5.5,10:21:32,7.7,2.3,10:26:59,6,0,0.0,0.0
7,10:37:53,4.7,10:45:23,12.2,7.5,10:50:06,7,0,0.0,0.0
8,10:54:52,4.8,11:05:31,15.4,10.6,11:10:18,8,0,0.0,0.0
9,11:33:39,3.3,11:38:16,8.0,4.6,11:41:36,9,0,107.3,10.7


In [7]:
print(statistics.mean(config.mean))
print(statistics.pvariance(config.mean))

11.58567106142685
0.4686181975124968


In [8]:
print(df['system_wait'].mean())
print(df['system_wait'].std())

11.585671061426849
3.8353416723852205


In [9]:
config = Object(
    starting_value = math.floor(df.loc[0, 'system_wait']),
    chunk_start = 0,
    total = [],
    count = [],
    estimation = [],
    total_temp = [],
    count_temp = [],
    estimation_temp = [])

def chunk1(config, data_slice, n):
    config.total_temp.extend([0]*n)
    config.count_temp.extend([0]*n)
    total = data_slice.sum()
    config.total.append(total)
    config.total_temp[-1] = total
    config.count.append(n) 
    config.count_temp[-1] = n
config.chunk_start = 0
for i in range(1, len(df)):
    if config.starting_value == math.floor(df.loc[i, 'system_wait']):
        data_slice = df.iloc[config.chunk_start:(i+1)]['system_wait']
        repetition_length = i - config.chunk_start
        chunk1(config, data_slice, repetition_length)
        config.chunk_start = i
data_slice = df.iloc[config.chunk_start:-1]['system_wait']
repetition_length = len(df) - config.chunk_start
chunk1(config, data_slice, repetition_length) 

total_mean = statistics.mean(config.total)
count_mean = statistics.mean(config.count)
def chunk2(config, data_slice, n, t, c, tm, cm):
    config.estimation_temp.extend([0]*n)
    estimation = n*tm/cm -((n-1)*(n*tm-t))/(n*cm-c)
    config.estimation.append(estimation)
    config.estimation_temp[-1] = estimation
config.chunk_start = 0
for i in range(1, len(df)):
    if config.starting_value == math.floor(df.loc[i, 'system_wait']):
        data_slice = df.iloc[config.chunk_start:(i+1)]['system_wait']
        repetition_length = i - config.chunk_start
        chunk2(config, data_slice, repetition_length, config.total_temp[i-1], config.count_temp[i-1], total_mean, count_mean)
        config.chunk_start = i
data_slice = df.iloc[config.chunk_start:-1]['system_wait']
repetition_length = len(df) - config.chunk_start
chunk2(config, data_slice, repetition_length, config.total_temp[-1], config.count_temp[-1], total_mean, count_mean)

df['repetition_total'] = config.total_temp
df['repetition_count'] = config.count_temp
df['repetition_estimation'] = config.estimation_temp
display(df)

,arrival_time,service_span,service_start,system_wait,queue_span,elimination_time,eliminated,queue_length,interval_total,interval_mean,repetition_total,repetition_count,repetition_estimation
0,08:05:29,8.9,08:06:29,9.9,1.0,08:15:24,,0,0.0,0.0,0.0,0,0.0
1,08:36:25,7.6,08:45:49,17.0,9.4,08:53:26,1,0,0.0,0.0,0.0,0,0.0
2,08:37:40,8.6,08:45:50,16.8,8.2,08:54:28,,1,0.0,0.0,0.0,0,0.0
3,09:06:33,2.4,09:07:40,3.5,1.1,09:10:05,2 3,0,0.0,0.0,56.4,4,12.1
4,09:32:22,2.9,09:38:37,9.1,6.2,09:41:29,4,0,0.0,0.0,0.0,0,0.0
5,09:44:45,4.3,09:48:01,7.6,3.3,09:52:19,5,0,0.0,0.0,0.0,0,0.0
6,10:19:15,5.5,10:21:32,7.7,2.3,10:26:59,6,0,0.0,0.0,0.0,0,0.0
7,10:37:53,4.7,10:45:23,12.2,7.5,10:50:06,7,0,0.0,0.0,0.0,0,0.0
8,10:54:52,4.8,11:05:31,15.4,10.6,11:10:18,8,0,0.0,0.0,0.0,0,0.0
9,11:33:39,3.3,11:38:16,8.0,4.6,11:41:36,9,0,107.3,10.7,0.0,0,0.0


In [10]:
print(statistics.mean(config.estimation))
print(statistics.pvariance(config.estimation))

11.649982970783418
0.15167456954991448


In [11]:
print(df['system_wait'].mean())
print(df['system_wait'].std())

11.585671061426849
3.8353416723852205
